## Crawling

- Khaiii: https://github.com/kakao/khaiii

* WHAT - Systematically browses
* WHY - Web indxing

- Scrapy -> 알아서 웹크롤링해주도록 두와주는 봇

크롤링 : http://example.webscraping.com/places/default/index

In [1]:
import requests
import time

def download(method, url,
             param=None, data=None,
            timeout=1, maxretries=3):
    try:
        resp = requests.request(method, url,
                     params=param, data=data,
                               headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500<= e.response.status_code <600\
        and maxretries >0:
            time.sleep(timeout)
            resp = download(method, url, param, data, timeout, maxretries-1)
            print("재시도")
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

In [2]:
from bs4 import BeautifulSoup

In [3]:
url = "http://example.webscraping.com/places/default/index/1"
html = download('get', url)

dom = BeautifulSoup(html.text, "lxml")
[_["href"] for _ in dom.find_all("a")]

['#',
 '/places/default/user/register?_next=/places/default/index/1',
 '/places/default/user/login?_next=/places/default/index/1',
 '/places/default/index',
 '/places/default/search',
 '/places/default/view/Argentina-11',
 '/places/default/view/Armenia-12',
 '/places/default/view/Aruba-13',
 '/places/default/view/Australia-14',
 '/places/default/view/Austria-15',
 '/places/default/view/Azerbaijan-16',
 '/places/default/view/Bahamas-17',
 '/places/default/view/Bahrain-18',
 '/places/default/view/Bangladesh-19',
 '/places/default/view/Barbados-20',
 '/places/default/index/0',
 '/places/default/index/2']

In [4]:
[_["href"] for _ in dom.find_all("a") if len(_["href"]) >3 ]

['/places/default/user/register?_next=/places/default/index/1',
 '/places/default/user/login?_next=/places/default/index/1',
 '/places/default/index',
 '/places/default/search',
 '/places/default/view/Argentina-11',
 '/places/default/view/Armenia-12',
 '/places/default/view/Aruba-13',
 '/places/default/view/Australia-14',
 '/places/default/view/Austria-15',
 '/places/default/view/Azerbaijan-16',
 '/places/default/view/Bahamas-17',
 '/places/default/view/Bahrain-18',
 '/places/default/view/Bangladesh-19',
 '/places/default/view/Barbados-20',
 '/places/default/index/0',
 '/places/default/index/2']

In [5]:
[requests.compat.urljoin(url, _["href"]) for _ in dom.find_all("a")if len(_["href"]) >3]

['http://example.webscraping.com/places/default/user/register?_next=/places/default/index/1',
 'http://example.webscraping.com/places/default/user/login?_next=/places/default/index/1',
 'http://example.webscraping.com/places/default/index',
 'http://example.webscraping.com/places/default/search',
 'http://example.webscraping.com/places/default/view/Argentina-11',
 'http://example.webscraping.com/places/default/view/Armenia-12',
 'http://example.webscraping.com/places/default/view/Aruba-13',
 'http://example.webscraping.com/places/default/view/Australia-14',
 'http://example.webscraping.com/places/default/view/Austria-15',
 'http://example.webscraping.com/places/default/view/Azerbaijan-16',
 'http://example.webscraping.com/places/default/view/Bahamas-17',
 'http://example.webscraping.com/places/default/view/Bahrain-18',
 'http://example.webscraping.com/places/default/view/Bangladesh-19',
 'http://example.webscraping.com/places/default/view/Barbados-20',
 'http://example.webscraping.com/

In [52]:
def parseURL (seed):
    
    html = download("get",seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(url, _["href"]) for _ in dom.find_all("a") 
            if _.has_attr("href") and len(_["href"]) >3] # and not startwith.(#) # has_attr('class') 속성이 있는지 확인 

In [53]:
queue = list()
# queue.append("https://www.google.com/search?newwindow=1&source=hp&ei=598rXbvWKY3e8wW4pZVI&q=cho&oq=cho&gs_l=psy-ab.3..35i39l2j0l8.7258.7500..7843...1.0..0.190.551.0j4......0....1..gws-wiz.....10..0i67.BLk6ausECnY")
queue.extend([_.find_parent()["href"] for _ in dom.select(".LC20lb")]) # ? seed 주소에 있는 걸 전부 탐색 
seen = list() # URL-seen 

while queue: # true
    baseURL = queue.pop(0) # pop(0) 앞에있는거, 나머지는 append니까 뒤에 붙을거  # -1로하면 제일 최근에 들어가는게 먼저나올 것이다. (FIFO, LIFO) -> 자료 구조
    seen.append(baseURL)
    time.sleep(30) # 너무 빠르게 리퀘스트 보내지 않도록 '슬립'이 포함되어야 한다. 
    linkList = parseURL(baseURL)
    for link in linkList:  # 이미 있음에도 중복되는 애들을 제거해주어야 한다. 
        if link not in queue and not link in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))
#     queue.extend(linkList)

# 실데이터를 다루면 tag a 를 다루더라도 href가 없다고 뜰 수 있다. 

Queue: 2, Seen: 1
Queue: 1, Seen: 2
Queue: 0, Seen: 3


- Seed 주소 : 씨앗되는 느낌

In [29]:
url = "https://www.google.com/search?newwindow=1&ei=seQrXam-F4Oa8wW_2IagCA&q=%EB%B0%95%EB%B3%B4%EC%98%81"

param = { 
    "q" : "박보영"}
html= download("get", url, param)
dom = BeautifulSoup(html.text,"lxml") 

In [32]:
len(dom.select(".LC20lb")) # seed 주소 8개 가져온거 처음 q는 8개

8

In [31]:
[_.find_parent()["href"] for _ in dom.select(".LC20lb")]

['https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81',
 'https://news.joins.com/article/22895953',
 'https://twitter.com/hashtag/%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://www.hani.co.kr/arti/culture/jejuand/861888.html',
 'https://www.hankyung.com/entertainment/article/201905030357H',
 'http://www.epochtimes.co.kr/news/articleView.html?idxno=415565',
 'http://sbsfune.sbs.co.kr/news/news_content.jsp?article_id=E10009477086',
 'http://ygp.co.kr/project/%EC%A2%8B%EC%9D%80%EB%8D%B0%EC%9D%B4-%EB%B0%95%EB%B3%B4%EC%98%81-ar-%ED%94%84%EB%A1%9C%EB%AA%A8%EC%85%98-%EC%98%81%EC%83%81/']

네이버에서 도메인 쓰면 도메인 내의 컨텐츠들만 수집 
focused crawling 

In [36]:
url = "https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=%EB%B0%95%EB%B3%B4%EC%98%81"

param = { "query":"박보영"}
html = download("get", url, param)
dom= BeautifulSoup(html.text, "lxml")

In [42]:
[_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title")]

['https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950',
 'https://asovc7777.blog.me/221548676988',
 'http://hshlog.com/221559890110',
 'https://blog.naver.com/js2y86?Redirect=Log&logNo=221530397718',
 'https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195']

In [38]:
[_ for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title")]

[<a class="sh_blog_title _sp_each_url _sp_each_title" href="https://blog.naver.com/imagetech1?Redirect=Log&amp;logNo=221530358950" onclick="return goOtherCR(this,'a=blg_1st*i.tit&amp;r=1&amp;i=90000003_0000000000000033943D04A6&amp;u='+urlencode(urlexpand(this.href)));" target="_blank" title="[어비스] 인물관계도 등장인물 몇부작 박보영 안효섭 드라마 어비스뜻 영혼소생구슬  tvN월화드라마">[어비스] 인물관계도 등장인물 몇부작 <strong class="hl">박보영</strong> 안효섭 드라마...</a>,
 <a class="sh_blog_title _sp_each_url _sp_each_title" href="https://asovc7777.blog.me/221548676988" onclick="return goOtherCR(this,'a=blg_1st*i.tit&amp;r=2&amp;i=90000003_00000000000000339554877C&amp;u='+urlencode(urlexpand(this.href)));" target="_blank" title="박보영 김영광 열애 !? ( 뽀누랑 영광이 정보 정리 ! )"><strong class="hl">박보영</strong> 김영광 열애 !? ( 뽀누랑 영광이 정보 정리 ! )</a>,
 <a class="sh_blog_title _sp_each_url _sp_each_title" href="http://hshlog.com/221559890110" onclick="return goOtherCR(this,'a=blg_1st*i.tit&amp;r=3&amp;i=90000003_000000000000003395FFA0BE&amp;u='+urlencode(urlexpand(th

In [43]:
requests.compat.urlparse("https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195")

ParseResult(scheme='https', netloc='blog.naver.com', path='/dlqlwm14', params='', query='Redirect=Log&logNo=221529807195', fragment='')

In [45]:
def checkBlog(url):
    return requests.compat.urlparse(url)[1] == "blog.naver.com" # notloc 

In [46]:
checkBlog("https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195")

True

In [49]:
[_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title") if checkBlog(_["href"])]

['https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950',
 'https://blog.naver.com/js2y86?Redirect=Log&logNo=221530397718',
 'https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195']

In [ ]:
# DB로 바꾸면 스키마를 어떻게 만들어야할지 생각해보기

In [65]:
def parseURL (seed):
    
    html = download("get",seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    if len(dom.select("#mainFrame")) < 1:  
        return[]
    
    seed = requests.compat.urljoin(seed, dom.select("#mainFrame")[0]["src"]) # documnet 다시 들어간 것 
    
    html = download("get", seed)
    dom= BeautifulSoup(html.text , "lxml")
    
    return [requests.compat.urljoin(seed, _["href"]) for _ in dom.find_all("a") 
            if _.has_attr("href") and len(_["href"]) > 3
           and checkBlog(requests.compat.urljoin(seed, _["href"]))] # and not startwith.(#) # has_attr('class') 속성이 있는지 확인 

In [66]:
queue = list()
# queue.append("https://www.google.com/search?newwindow=1&source=hp&ei=598rXbvWKY3e8wW4pZVI&q=cho&oq=cho&gs_l=psy-ab.3..35i39l2j0l8.7258.7500..7843...1.0..0.190.551.0j4......0....1..gws-wiz.....10..0i67.BLk6ausECnY")
# queue.extend([_.find_parent()["href"] for _ in dom.select(".LC20lb")]) # ? seed 주소에 있는 걸 전부 탐색 
queue.extend([_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title") if checkBlog(_["href"])])
# print(queue)
seen = list() # URL-seen 

while queue: # true
    baseURL = queue.pop(0) # pop(0) 앞에있는거, 나머지는 append니까 뒤에 붙을거  # -1로하면 제일 최근에 들어가는게 먼저나올 것이다. (FIFO, LIFO) -> 자료 구조
    seen.append(baseURL)
    time.sleep(1) # 너무 빠르게 리퀘스트 보내지 않도록 '슬립'이 포함되어야 한다. 
    linkList = parseURL(baseURL)
    for link in linkList:  # 이미 있음에도 중복되는 애들을 제거해주어야 한다. 
        if link not in queue and not link in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))
#     queue.extend(linkList)

# 실데이터를 다루면 tag a 를 다루더라도 href가 없다고 뜰 수 있다. 

['https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950', 'https://blog.naver.com/js2y86?Redirect=Log&logNo=221530397718', 'https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195']
Queue: 17, Seen: 1
Queue: 32, Seen: 2
Queue: 48, Seen: 3
Queue: 47, Seen: 4
Queue: 46, Seen: 5
Queue: 45, Seen: 6
Queue: 44, Seen: 7
Queue: 43, Seen: 8
Queue: 42, Seen: 9
Queue: 41, Seen: 10
Queue: 40, Seen: 11
Queue: 39, Seen: 12
Queue: 38, Seen: 13
Queue: 37, Seen: 14
Queue: 64, Seen: 15
Queue: 63, Seen: 16
404
Not Found
Queue: 62, Seen: 17
Queue: 61, Seen: 18
Queue: 60, Seen: 19
Queue: 59, Seen: 20
Queue: 58, Seen: 21
Queue: 57, Seen: 22
Queue: 56, Seen: 23
Queue: 55, Seen: 24
Queue: 54, Seen: 25
Queue: 53, Seen: 26
Queue: 52, Seen: 27
Queue: 51, Seen: 28
Queue: 50, Seen: 29
Queue: 49, Seen: 30
Queue: 48, Seen: 31
Queue: 64, Seen: 32
Queue: 63, Seen: 33
Queue: 62, Seen: 34
Queue: 61, Seen: 35
Queue: 60, Seen: 36
Queue: 59, Seen: 37
Queue: 58, Seen: 38
Queue: 57, Seen: 39
Queue: 56, Se

KeyboardInterrupt: 

- Daum focused Crawling 

In [68]:
url = "https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&q=%EB%B0%95%EB%B3%B4%EC%98%81"

param = { "query":"박보영"}
html = download("get", url, param)
dom= BeautifulSoup(html.text, "lxml")

In [95]:
[_["href"] for _ in dom.select("#blogColl .f_link_b")]

['http://adam24eve.tistory.com/858',
 'http://cscs0717.tistory.com/27',
 'http://listup.tistory.com/248',
 'http://34344541.tistory.com/43']

In [99]:
requests.compat.urlparse("https://adam24eve.tistory.com/858")

ParseResult(scheme='https', netloc='adam24eve.tistory.com', path='/858', params='', query='', fragment='')

In [109]:
def checkBlog(url):
    return requests.compat.urlparse(url)[1].endswith("tistory.com") # notloc 

In [112]:
[_["href"] for _ in dom.select("#blogColl .f_link_b") if checkBlog(_["href"])]

['http://adam24eve.tistory.com/858',
 'http://cscs0717.tistory.com/27',
 'http://listup.tistory.com/248',
 'http://34344541.tistory.com/43']

In [113]:
def parseURL (seed):

    html = download("get", seed)
    dom= BeautifulSoup(html.text , "lxml")
    
    return [requests.compat.urljoin(seed, _["href"]) for _ in dom.find_all("a") 
            if _.has_attr("href") and len(_["href"]) > 3
           and checkBlog(requests.compat.urljoin(seed, _["href"]))] # and not startwith.(#) # has_attr('class') 속성이 있는지 확인 

In [114]:
queue = list()
queue.extend([_["href"] for _ in dom.select("#blogColl .f_link_b") if checkBlog(_["href"])])
seen = list() # URL-seen 

while queue: # true
    baseURL = queue.pop(0) # pop(0) 앞에있는거, 나머지는 append니까 뒤에 붙을거  # -1로하면 제일 최근에 들어가는게 먼저나올 것이다. (FIFO, LIFO) -> 자료 구조
    seen.append(baseURL)
    time.sleep(1) # 너무 빠르게 리퀘스트 보내지 않도록 '슬립'이 포함되어야 한다. 
    linkList = parseURL(baseURL)
    for link in linkList:  # 이미 있음에도 중복되는 애들을 제거해주어야 한다. 
        if link not in queue and not link in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))
#     queue.extend(linkList)

# 실데이터를 다루면 tag a 를 다루더라도 href가 없다고 뜰 수 있다. 

Queue: 44, Seen: 1
Queue: 60, Seen: 2
Queue: 83, Seen: 3
Queue: 90, Seen: 4
Queue: 96, Seen: 5
Queue: 102, Seen: 6
Queue: 108, Seen: 7
Queue: 111, Seen: 8
Queue: 118, Seen: 9
Queue: 121, Seen: 10
Queue: 120, Seen: 11
Queue: 119, Seen: 12
Queue: 118, Seen: 13
Queue: 117, Seen: 14
Queue: 116, Seen: 15
Queue: 115, Seen: 16
Queue: 116, Seen: 17
Queue: 115, Seen: 18
Queue: 114, Seen: 19
Queue: 113, Seen: 20


KeyboardInterrupt: 

- link도 depth를 줘서 더이상 가지가 뻗어나가지 않도록 짤라주는 코롤링 (예상 가능한 범위 안에서만 뽑기) 

In [121]:
def parseURL (seed):

    html = download("get", seed[0])
    dom= BeautifulSoup(html.text , "lxml")
    
    return [(requests.compat.urljoin(seed[0], _["href"]), seed[1]+1) for _ in dom.find_all("a") # seed[1]+1 가지치기 한 것 
            if seed[1] < 2 and _.has_attr("href") and len(_["href"]) > 3
           and checkBlog(requests.compat.urljoin(seed[0], _["href"]))] # has_attr('class') 속성이 있는지 확인 

In [122]:
queue = list()
queue.extend([(_["href"], 1) for _ in dom.select("#blogColl a.f_link_b") if checkBlog(_["href"])])  # tuple로 준다. 
seen = list() # URL-seen 

while queue: # true
    baseURL = queue.pop(0) # pop(0) 앞에있는거, 나머지는 append니까 뒤에 붙을거  # -1로하면 제일 최근에 들어가는게 먼저나올 것이다. (FIFO, LIFO) -> 자료 구조
    seen.append(baseURL)
    time.sleep(1) # 너무 빠르게 리퀘스트 보내지 않도록 '슬립'이 포함되어야 한다. 
    linkList = parseURL(baseURL)
    for link in linkList:  # 이미 있음에도 중복되는 애들을 제거해주어야 한다. 
        if link not in queue and not link in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))
#     queue.extend(linkList)

# 실데이터를 다루면 tag a 를 다루더라도 href가 없다고 뜰 수 있다. 

Queue: 45, Seen: 1
Queue: 61, Seen: 2
Queue: 85, Seen: 3
Queue: 92, Seen: 4
Queue: 91, Seen: 5
Queue: 90, Seen: 6
Queue: 89, Seen: 7
Queue: 88, Seen: 8
Queue: 87, Seen: 9
Queue: 86, Seen: 10
Queue: 85, Seen: 11
Queue: 84, Seen: 12
Queue: 83, Seen: 13
Queue: 82, Seen: 14
Queue: 81, Seen: 15
Queue: 80, Seen: 16
Queue: 79, Seen: 17
Queue: 78, Seen: 18
Queue: 77, Seen: 19
Queue: 76, Seen: 20
Queue: 75, Seen: 21
Queue: 74, Seen: 22
Queue: 73, Seen: 23
Queue: 72, Seen: 24
Queue: 71, Seen: 25
Queue: 70, Seen: 26
Queue: 69, Seen: 27
Queue: 68, Seen: 28
Queue: 67, Seen: 29
Queue: 66, Seen: 30
Queue: 65, Seen: 31
Queue: 64, Seen: 32
Queue: 63, Seen: 33
Queue: 62, Seen: 34
Queue: 61, Seen: 35
Queue: 60, Seen: 36
Queue: 59, Seen: 37
Queue: 58, Seen: 38
Queue: 57, Seen: 39
Queue: 56, Seen: 40
Queue: 55, Seen: 41
Queue: 54, Seen: 42
Queue: 53, Seen: 43
Queue: 52, Seen: 44
Queue: 51, Seen: 45
Queue: 50, Seen: 46
Queue: 49, Seen: 47
Queue: 48, Seen: 48
Queue: 47, Seen: 49
Queue: 46, Seen: 50
Queue: 45

In [124]:
import re
re.search(".+?\.tistory.com", "http://adam24eve.tistory.com/858")

<re.Match object; span=(0, 28), match='http://adam24eve.tistory.com'>

- DB로 바꿀 것 

- 스키마를 몇개로 해야할까? 
- entity가 몇개 나오면 좋을까? 

링크만 뽑았지 본문내용을 저장을 안했다. 

- 웹수집기 (crawler) - 특정 도메인에 있는 애들만 가져올 줄 알아야한다. 

* DB 만들기
- 문자열에다가 PK 값 쓰면 안된다. 
- pk 값을 따로 만들어야한다. (ID) 
- url: distinct value

In [ ]:
# 테이블 하나에 있는 변수들 
table1 = id, path, param, seen, date 
table2 = id, netloc, date  # focused data - 특정 도메인만 써주면 된다.
# id = pk 
url = netloc + path + param 
# DBMS -> 원자성: entity 중복 최소화하면서 쪼갤 수 있을 만큼 쪼개야한다. 

In [1]:
import sqlite3

In [133]:
con = sqlite3.connect("bot.db")
cur = con.cursor() # cursor? 

In [188]:
# param 은 null일 수도 있따.
cur.executescript("""
    DROP TABLE IF EXISTS table1; 
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table2_id INTEGER NOT NULL,
        path TEXT NOT NULL,
        param TEXT,
        seen BOOLEAN DEFAULT FALSE NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        netloc TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
""")
con.commit()

In [167]:
requests.compat.urlparse(html.request.url)

ParseResult(scheme='https', netloc='www.google.com', path='/search', params='', query='q=%EB%B0%95%EB%B3%B4%EC%98%81', fragment='')

In [170]:
# Seed URLs - DB Insert 
url = "https://www.google.com/search"
html = download("get", url, param = {"q": "박보영"})
dom= BeautifulSoup(html.text, "lxml")

for href in [_.find_parent()["href"] for _ in dom.select(".LC20lb")]: 
    _urlparse = requests.compat.urlparse(href)
    netloc = "://".join(_urlparse[:2]) # 1: 스키마 2: noetloc
    cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc]) 
    netlocID = cur.fetchone()
    if not netlocID: 
        cur.execute("INSERT INTO table2(netloc) VALUES(?)", [netloc])
        con.commit() # 추가하면 반영해야한다. 
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc]) 
        netlocID = cur.fetchone()
        
    cur.execute("INSERT INTO table1(table2_id, path, param) VALUES(?,?,?)", [netlocID[0], _urlparse[2], _urlparse[4]]) # type 맞춰줘야한다. 
    con.commit()
#     print(cur.lastrowid, netlocID)
    
# DB Select Limit 0,1 

In [175]:
while True:
    cur.execute("""
                SELECT table1.id, table2.netloc, 
                       table1.path, table1.param 
                FROM table1 
                JOIN table2 
                    ON table1.table2_id=table2.id 
                WHERE table1.seen=FALSE
                ORDER BY table1.date ASC 
                LIMIT 0, 1;
    """)
    seed = cur.fetchone()
    if not seed:
        break;
    print(seed)
    print("{0}{1}?{2}".format(seed[1], seed[2], seed[3]))
    break

(1, 'https://namu.wiki', '/w/%EB%B0%95%EB%B3%B4%EC%98%81', '')
https://namu.wiki/w/%EB%B0%95%EB%B3%B4%EC%98%81?


In [187]:
def parseURL(seed):
    
    html = download("get", seed)
    dom= BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(seed, _["href"]) for _ in dom.find_all("a") # seed[1]+1 가지치기 한 것 
            if _.has_attr("href") and len(_["href"]) > 3]

In [186]:
i = 0
while True:
    cur.execute("""
                SELECT table1.id, table2.netloc, 
                       table1.path, table1.param 
                FROM table1 
                JOIN table2 
                    ON table1.table2_id=table2.id 
                WHERE table1.seen=FALSE
                ORDER BY table1.date ASC 
                LIMIT 0, 1;
    """)
    seed = cur.fetchone()
    if not seed or i >1000 :
        break;
    i += 1
    cur.execute("""
        UPDATE table1
        SET seen=TRUE
        WHERE id=?
    """, [seed[0]])
    con.commit()
        
    baseURL = "{0}{1}?{2}".format(seed[1], seed[2], seed[3])
    for href in parseURL(baseURL):
        _urlparse = requests.compat.urlparse(href)
        netloc = "://".join(_urlparse[:2]) # 1: 스키마 2: noetloc
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc]) 
        netlocID = cur.fetchone()
        if not netlocID: 
            cur.execute("INSERT INTO table2(netloc) VALUES(?)", [netloc])
            con.commit() # 추가하면 반영해야한다. 
            cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc]) 
            netlocID = cur.fetchone()
        cur.execute("INSERT INTO table1(table2_id, path, param) VALUES(?,?,?)", [netlocID[0], _urlparse[2], _urlparse[4]])
        con.commit() # q 생성 

KeyboardInterrupt: 

In [207]:
# param 은 null일 수도 있따.
cur.executescript("""
    DROP TABLE IF EXISTS table1; 
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table2_id INTEGER NOT NULL,
        path TEXT NOT NULL,
        param TEXT,
        depth INTEGER NOT NULL,
        inbound INTEGER NOT NULL, 
        seen BOOLEAN DEFAULT FALSE NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        netloc TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
""")
con.commit()

In [208]:
#seed urls -> dbinsert

url = "https://www.google.com/search"
html = download("get", url, param={"q":"박보영"})
dom = BeautifulSoup(html.text, "lxml")

for href in [_.find_parent()["href"]
             for _ in dom.select(".LC20lb")]:
    
    _urlparse = requests.compat.urlparse(href)
    netloc = "://".join(_urlparse[:2])
    cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc]) #netloc을 시퀀스로 만들어서 넘겨줘야함
    
    netlocID = cur.fetchone()
    if not netlocID:
        cur.execute("INSERT INTO table2(netloc) VALUES(?)", [netloc])
        
        con.commit()
        
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc]) #netloc을 시퀀스로 만들어서 넘겨줘야함
        
        netlocID = cur.fetchone()
        # table2에 집어넣기
        
    cur.execute("INSERT INTO table1(table2_id, path, param, depth, inbound) VALUES(?,?,?,?,?)", [netlocID[0], _urlparse[2], _urlparse[4], 1, 0])
    con.commit()
    print(cur.lastrowid, netlocID)

#db select limit 0,1

1 (1,)
2 (2,)
3 (2,)
4 (3,)
5 (4,)
6 (5,)
7 (6,)
8 (7,)
9 (8,)


In [ ]:
while True:
    cur.execute("""
                SELECT table1.id, table2.netloc, 
                       table1.path, table1.param, table1.depth, table2.id
                FROM table1 
                JOIN table2 
                    ON table1.table2_id=table2.id 
                WHERE table1.seen=FALSE and table1.depth < 3
                ORDER BY table1.date ASC 
                LIMIT 0, 1;
    """)
    seed = cur.fetchone()
    if not seed :
        break;

    cur.execute("""
        UPDATE table1
        SET seen=TRUE
        WHERE id=?
    """, [seed[0]])
    con.commit()
        
    baseURL = "{0}{1}?{2}".format(seed[1], seed[2], seed[3])
    for href in parseURL(baseURL):
        _urlparse = requests.compat.urlparse(href)
        netloc = "://".join(_urlparse[:2]) # 1: 스키마 2: noetloc
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc]) 
        netlocID = cur.fetchone()
        if not netlocID: 
            cur.execute("INSERT INTO table2(netloc) VALUES(?)", [netloc])
            con.commit() # 추가하면 반영해야한다. 
            cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc]) 
            netlocID = cur.fetchone()
        cur.execute("INSERT INTO table1(table2_id, path, param, depth, inbound) \
        VALUES(?,?,?,?,?)", [netlocID[0], _urlparse[2], _urlparse[4], seed[4]+1, seed[5]]) # depth 3 까지만 가져오기 
        con.commit() # q 생성  

In [210]:
select inbound, count(inbound)
from table 1 where table2_id=2 group by inbound; 

SyntaxError: invalid syntax (<ipython-input-210-0cb8bd3eade5>, line 1)

In [224]:
cur.execute("""
    select inbound, count(inbound)
    from table1 
    where table2_id=2 group by inbound;
""")
cur.fetchall() # id= 10 1번링크 587번 inbound 된것, 자기 링크에서 1310 inbound 된 것  

[(0, 2), (1, 2), (2, 1375)]

In [218]:
cur.execute("""
    select inbound, count(inbound)
    from table1 
    where inbound=1;
""")
cur.fetchall() # 1의 링크가 사용된 개수 11332번 

[(1, 11332)]

In [ ]:
# 컨텐츠를 넣어줄 것이다. 

In [ ]:
cur.close()
con.close()